# Understand embeddings with Word2Vec

### Exercise objectives:
- Convert 🔠 words to 🔢 vector representations thanks to embeddings
- Discover the powerful Word2Vec algorithm

<hr>

_Embeddings_ are representations of words using vectors. These embeddings can be learned within a Neural Network. But it can take time to converge. Another option is to learn them as a first step. Then, use them directly to feed the word representations into a Recurrent Neural Network. 

▶️ Run this cell and make sure the version of 📚 [Gensim - Word2Vec](https://radimrehurek.com/gensim/auto_examples/index.html) you are using is ≥ 4.0!

In [1]:
!pip freeze | grep gensim

gensim==4.2.0


In [2]:
!pip freeze | grep tensorflow

tensorflow-cpu==2.10.0
tensorflow-datasets==4.6.0
tensorflow-estimator==2.10.0
tensorflow-io-gcs-filesystem==0.27.0
tensorflow-metadata==1.10.0


# The data

Keras provides many datasets, among which is the IMDB dataset 🎬:
- It is comprised of sentences that are ***movie reviews***. 
- Each of these reviews is related to a score given by the reviewer.

❓ **Question** ❓ First of all, let's load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer can handle it. Otherwise, rerun with a lower number.  

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy. 

In [3]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

2024-11-28 12:53:59.978969: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-28 12:54:06.331696: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteVZZHZ0/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteVZZHZ0/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


2024-11-28 12:54:33.781046: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<b><u>Embeddings in the previous challenge</u></b>:

In the previous exercise, we jointly learned a representation for the words, and fed this representation to a RNN, as shown down below 👇: 

<img src="layers_embedding.png" width="400px" />

However, this increases the number of parameters to learn, which slows down and increases the difficulty of convergence!

<b><u>Embeddings in the current challenge</u></b>:

For this reason, we will separate the steps of learning the word representation and feeding it into a RNN. As shown here: 

<img src="word2vec_representation.png" width="400px" />

We will learn the embedding with Word2Vec.

The drawback is indeed that the learned embeddings are not _specifically_ designed for our task. However, learning them independently of the task at hand (sentiment analysis) has some advantages: 
- it is very fast to do in general (with Word2Vec)
- the representation learned by Word2Vec is still meaningful 
- the convergence of the RNN alone will be easier and faster

So let's learn an embedding with Word2Vec and see how meaningful it is!

# Embedding with Word2Vec

Let's use Word2Vec to embed the words of our sentences. Word2Vec will be able to convert each word to a fixed-size vectorial representation.

For instance, we will have:
- 🐶 _dog_ $\rightarrow$ [0.1, -0.3, 0.8]
- 🐱 _cat_ $\rightarrow$ [-1.1, 2.3, 0.7]
- 🍏 _apple_ $\rightarrow$ [3.1, 0.9, -4.7]

Here, your embedding space is of size 3.

***What is a "good" numerical representation of words?***

- ***Words with close meanings should be geometrically close in your embedding space!***

    - Look at the following example which represents a bi-dimensional embedding space.

![Embedding](word_embedding.png)

❓ **Question** ❓ Let's run Word2Vec! 

[📚 **Gensim**](https://radimrehurek.com/gensim/)  is a great Python package that makes the use of the Word2Vec algorithm easy to implement, fast and accurate (which is not an easy task!).

1. The following code imports Word2Vec from Gensim. 

2. The second line learns the embedding representation of the words thanks to the sentences in `X_train`. 
3. The third line stores the words and their trained embeddings in `wv`.

NOTA: VALOR POR DEFECTO DEL VECTOR SIZE = 100!!

In [4]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)
wv = word2vec.wv

In [34]:
word2vec.vector_size

100

Let's look at the embedded representation of some words.

You can use `wv` as a dictionary.
For instance, `wv['dog']` will return a representation of `dog` in the embedding space.

In [5]:
wv['dog']

array([-0.20469321,  0.13796845, -0.09916382,  0.20162688, -0.06696376,
       -0.3202459 ,  0.04004217,  0.53110677, -0.19716196, -0.13077085,
       -0.01055434, -0.3233028 , -0.00941597,  0.08480119, -0.04713192,
       -0.23816267,  0.15412113, -0.24068962, -0.00372023, -0.31879377,
        0.13616636,  0.12143338,  0.23483503, -0.06922397, -0.07315948,
        0.07520544, -0.14190802, -0.0297297 , -0.14883077,  0.02459871,
        0.15901339,  0.00627453,  0.1032757 , -0.34991413, -0.16786288,
        0.13806084,  0.17685926, -0.1893009 , -0.25130337, -0.21867082,
       -0.1148202 , -0.24066809, -0.247676  ,  0.14106525,  0.32434478,
       -0.08277486, -0.13017736, -0.02455089,  0.22560929,  0.16186748,
        0.0846026 , -0.184851  , -0.20960738, -0.08645915, -0.24006921,
        0.17612296,  0.13098213,  0.00417137, -0.16307293,  0.101619  ,
        0.11712223, -0.01554456,  0.01175048,  0.09610549, -0.21689668,
        0.25843903, -0.03951712,  0.27957934, -0.3598501 ,  0.31

❓ **Question** ❓ Try different words - especially, try non-existing words to see that they don't have any representation (which is perfectly normal as their representation was not learned). 

In [9]:
wv['kaput']

KeyError: "Key 'kaput' not present"

❓ **Question** ❓ What is the size of each word representation, and therefore, what is the size of the embedding space?

In [12]:
len(wv['dog'])

100

In [10]:
wv['dog'].shape

(100,)

🧐 How do we know whether this embedding make any sense or not? 

💡 To investigate this question, we will check that words with a close meaning have close representations. 

👉 Let's use the [**`Word2Vec.wv.most_similar`**](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.most_similar) method that, given an input word, displays the "closest" words in the embedding space. If the embedding is well done, then words with similar meanings will have similar representation in the embedding space.

❓ **Question** ❓ Try out the `most_similar` method on different words. 

🧑🏿‍🏫 The quality of the closeness will depend on the quality of your embedding, and thus, depend on the number of sentences that you have loaded and from which you create your embedding.

In [13]:
wv.most_similar('dog', topn=5)

[('crying', 0.9883103370666504),
 ('sought', 0.9875913858413696),
 ('offered', 0.9868466258049011),
 ('alien', 0.9856255650520325),
 ('orders', 0.9854063987731934)]

In [14]:
wv.most_similar('cat', topn=5)

[('boss', 0.9893953204154968),
 ('colonel', 0.9880110621452332),
 ('revenge', 0.9879565238952637),
 ('marriage', 0.9861485958099365),
 ('plane', 0.9856883883476257)]

In [15]:
wv.most_similar('plane', topn=5)

[('boss', 0.9888747930526733),
 ('birth', 0.9885120391845703),
 ('apartment', 0.9883167147636414),
 ('soldier', 0.9873052835464478),
 ('photo', 0.9860789775848389)]

In [16]:
wv.most_similar('mexico', topn=5)

[('fairy', 0.9903624057769775),
 ('ultimate', 0.9900732636451721),
 ('shadow', 0.9900674819946289),
 ('force', 0.9892361760139465),
 ('flying', 0.9882334470748901)]

📚 Similarly to `most_similar` used on words directly, we can use [**`similar_by_vector`**](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.similar_by_vector) on vectors to do the same thing:

In [17]:
wv.similar_by_vector('dog', topn=5)

[('crying', 0.9883103370666504),
 ('sought', 0.9875913858413696),
 ('offered', 0.9868466258049011),
 ('alien', 0.9856255650520325),
 ('orders', 0.9854063987731934)]

In [18]:
wv.similar_by_vector('cat', topn=5)

[('boss', 0.9893953204154968),
 ('colonel', 0.9880110621452332),
 ('revenge', 0.9879565238952637),
 ('marriage', 0.9861485958099365),
 ('plane', 0.9856883883476257)]

In [19]:
wv.similar_by_vector('plane', topn=5)

[('boss', 0.9888747930526733),
 ('birth', 0.9885120391845703),
 ('apartment', 0.9883167147636414),
 ('soldier', 0.9873052835464478),
 ('photo', 0.9860789775848389)]

In [20]:
wv.similar_by_vector('mexico', topn=5)

[('fairy', 0.9903624057769775),
 ('ultimate', 0.9900732636451721),
 ('shadow', 0.9900674819946289),
 ('force', 0.9892361760139465),
 ('flying', 0.9882334470748901)]

# Arithmetic on words

Now, let's perform some mathematical operations on words, i.e. on their vector representations!

As any learned word is represented as a vector, you can do basic arithmetic operations, such as:

$$W2V(good) - W2V(bad)$$

❓ **Question** ❓ Do this mathematical operation and print the result

In [21]:
wv['good'] - wv['bad']

array([-0.38642424, -0.3049647 ,  0.37812942,  0.14260566,  0.18687874,
       -0.15963626, -0.35334617, -0.04783502, -0.04213536, -0.11095375,
        0.17767927,  0.15039355, -0.01636046, -0.11615023,  0.1438353 ,
        0.59536624, -0.5425384 ,  0.16566634, -0.33138457,  0.19923413,
       -0.18845415,  0.0529099 ,  0.01611316,  0.0847289 , -0.05017525,
       -0.07465136, -0.03586718,  0.8668345 ,  0.01826268, -0.6490824 ,
       -0.4347608 ,  0.04829777,  0.07341397,  0.39611673,  0.24615866,
       -0.6939299 ,  0.40610486, -0.16533032, -0.56204945,  0.64325976,
       -0.22377586, -0.15598857, -0.5053953 , -0.23313707,  0.37681943,
       -0.20190218,  0.38768023,  0.03074253, -0.3565907 ,  0.62973315,
       -0.08614379, -0.01320046, -0.42175347,  0.09327859,  0.10734764,
        0.1483708 , -0.24199423,  0.26287848,  0.08085388,  0.67502934,
        0.02557592,  0.21924764, -0.23843849, -0.36307198,  0.07395732,
        0.1531161 , -0.37954465,  0.01717436,  0.40825534,  0.60

Now, imagine for a second that the following equality holds true:

$$W2V(good) - W2V(bad) = W2V(nice) - W2V(stupid)$$

which is equivalent to:

$$W2V(good) - W2V(bad) + W2V(stupid) = W2V(nice)$$

❓ **Question** ❓ Let's, just for fun (as it would be bold of us to think that this equality holds true ...), do the operation $W2V(good) - W2V(bad) + W2V(stupid)$ and store it in a `res` variable (which will be a vector of size 100 that you can print).

In [23]:
res = wv['good'] - wv['bad'] + wv['stupid']
res

array([-0.14624283, -0.39411464,  0.506402  , -0.18664372,  0.30105346,
       -0.699857  , -0.5169113 ,  0.36429325, -0.5013727 , -0.4246928 ,
       -0.07264626, -0.28169906,  0.16333216,  0.17575786,  0.35068488,
        0.22967073, -0.15690258, -0.43160444, -0.53021014, -0.7292113 ,
        0.32602662,  0.22632591,  0.57150275, -0.04712141, -0.18460095,
       -0.23006484, -0.3802933 ,  0.725169  , -0.24339917, -0.61430514,
        0.20829648,  0.08057189,  0.38218677, -0.04748482, -0.1926673 ,
        0.20548499,  0.39665726, -0.07771731, -0.81191266, -0.3793137 ,
       -0.22507265, -0.7312472 , -0.34683877,  0.34245223,  0.605355  ,
       -0.06191586, -0.07613313, -0.43454343, -0.397311  ,  0.7236783 ,
        0.09090437, -0.32563958, -0.45690545,  0.11817419,  0.14653242,
        0.45899063,  0.0289171 ,  0.19661084, -0.46315306,  0.28416765,
        0.11043123, -0.02344511,  0.15329236, -0.01383567, -0.5966545 ,
        0.6621892 , -0.29342678,  0.11704759,  0.106837  ,  0.79

In [24]:
res.shape

(100,)

We said earlier, that for any vector it is possible to see the closest vectors in the embedding space.

❓ **Question** ❓ Look at the closest vectors of `res`

💡 _Hint_: `similar_by_vector`

In [25]:
wv.similar_by_vector(res, topn=5)

[('good', 0.8083450198173523),
 ('nice', 0.7830855846405029),
 ('great', 0.7613089084625244),
 ('trick', 0.7441134452819824),
 ('also', 0.7396683692932129)]

Incredible right! You can do arithmetic operations on words!

❓ **Question** ❓ You can try on arithmetic such as 

$$W2V(Boy) - W2V(Girl) = W2V(Man) - W2V(Woman)$$

or 

$$W2V(Queen) - W2V(King) = W2V(actress) - W2V(actor)$$

❗ **Remark** ❗ You will probably see that the results are not perfect. But don't forget that you trained your model on a very small corpus.

Las operaciones en el espacio de embedding funcionan como álgebra vectorial, y puedes decir que estás "despejando" para encontrar un valor resultante. Ojo: no necesitas realizar un despeje literal como en matemáticas, sino que simplemente calculas el vector resultante y luego buscas las palabras más cercanas a ese vector.
wv(’man’)=wv(’boy’)−wv(’girl’)+wv(’woman’)

In [27]:
res_man = wv['boy'] - wv['girl'] + wv['woman']
res_man

array([-3.43390614e-01,  2.26821512e-01, -3.03867072e-01,  6.80109799e-01,
        2.59119928e-01, -3.90334547e-01,  2.44505271e-01,  9.34364676e-01,
       -2.48710752e-01, -6.57366872e-01, -1.82230920e-02, -3.63611668e-01,
       -5.53929508e-02,  4.43315208e-02,  2.99609154e-02,  4.36584093e-02,
        6.04711920e-02,  1.98628351e-01, -8.71861354e-03, -4.94039237e-01,
       -2.47485638e-02,  1.03332259e-01,  2.03679234e-01, -4.96321946e-01,
        1.93227753e-02,  2.99409144e-02, -7.35979557e-01,  2.20944211e-01,
       -2.58724630e-01, -2.40152776e-02, -2.24539056e-01, -1.35948375e-01,
        1.71710208e-01, -4.86492753e-01, -1.80249900e-01,  6.82955384e-02,
        2.63315260e-01, -2.44488716e-01, -3.23137611e-01,  1.70911849e-02,
       -1.33510143e-01, -3.22919548e-01, -8.61149788e-01,  3.12164426e-01,
        2.98718333e-01, -2.70988882e-01,  1.11500382e-01, -1.93865210e-01,
        7.37626553e-01,  4.85783637e-01, -5.74878603e-02, -4.42680299e-01,
       -2.13356167e-01,  

In [28]:
wv.similar_by_vector(res_man, topn=5)

[('boy', 0.9503494501113892),
 ('oscar', 0.9249392151832581),
 ('child', 0.9249180555343628),
 ('woman', 0.9226078987121582),
 ('emmy', 0.9209305644035339)]

In [31]:
res_actress = wv['queen'] - wv['king'] + wv['actor']
res_actress

array([-0.37531477,  0.10925092,  0.1736308 ,  0.10411971,  0.2390885 ,
       -0.3462969 , -0.1217375 ,  0.378987  , -0.2898801 , -1.0983377 ,
       -0.35948765, -0.20948146, -0.5681345 , -0.56117296,  0.02829057,
        0.14684245, -0.1190138 ,  0.03280418, -0.2669912 , -0.40599367,
       -0.19518773,  0.43165678,  0.1525361 , -0.13762206, -0.48569447,
        0.03216822, -0.4797981 ,  0.10230601, -0.05471544, -0.01644239,
       -0.13342804, -0.23204708,  0.12764236, -0.46921   , -0.0491358 ,
        0.29269642,  0.3165304 , -0.13844512,  0.16573171,  0.20358641,
        0.282178  , -0.7377386 , -0.5152957 ,  0.9700879 ,  0.3005782 ,
       -0.7191129 ,  0.41561615, -0.5034434 ,  0.7015619 ,  0.38273597,
        0.01503023, -0.7387305 , -0.35172454,  0.11794586,  0.1259969 ,
        1.204424  ,  0.46033478, -0.45742208,  0.09895664,  0.422045  ,
        0.13450785, -0.00722295,  0.92361176,  0.00937673, -0.2194359 ,
        0.50862473,  0.02355043,  0.52856404, -0.41907465, -0.46

In [32]:
wv.similar_by_vector(res_actress, topn=5)

[('actor', 0.9697166681289673),
 ('performance', 0.884698212146759),
 ('role', 0.8725627660751343),
 ('actress', 0.8584088683128357),
 ('job', 0.836007833480835)]

<u><i>Some notes about Word2Vec as an internal Neural Network</i></u>:

You might wonder where does this magic comes from (at quite a low price, you just ran a line of code on a very small corpus and it was trained within few minutes). The magic comes from the way Word2Vec is trained. The details are quite complex, but you can remember that Word2vec, in `word2vec = Word2Vec(sentences=X_train)`, actually trains a internal neural network (that you don't see).  

In a nutshell, this internal neural network predicts a word from the surroundings words in a sentences. Hence, it splits the original sentences, then for each split it chooses some words as inputs $X$ and a word as the output $y$ which it tries to predict, using the embedding space.

And as with any neural network, Word2Vec has some hyperparameters. Let's play with some of these. 

# Word2Vec hyperparameters

❓ **Question** ❓ The first important hyperparameter is the `vector_size` argument. It corresponds to the size of the embedding space. Learn a new `word2vec_2` model, still trained on the `X_train`, but with a smaller or higher `vector_size`.

Verify on some words that the embedding size is the one you chose.

In [35]:
word2vec_2 = Word2Vec(sentences=X_train, vector_size=50)
wv_2 = word2vec_2.wv

In [36]:
len(wv_2['dog'])

50

❓ **Question** ❓ Use the **`Word2Vec.wv.key_to_index`** attribute to display the size of the learned vocabulary. Compare it to the number of different words in `X_train`.

In [ ]:
AGEGUEMOS LEN POR LONGITUD

In [38]:
len(wv_2.key_to_index)

8006

In [39]:
len(set(word for sentence in X_train for word in sentence))

30419

There is an important difference between the number of words in the train sentences and in the Word2Vec vocabulary, even though it has been trained on the train sentence set. The reasons comes from the second important hyperparameter of Word2Vec:  `min_count`. 

`min_count` is a integer that tells you how many occurrences a given word should have to be learned in the embedding space. For instance, let's say that the word "movie" appears 1000 times in the corpus and "simba" only 2 times. If `min_count=3`, the word "simba" will be skipped during the training.

The intention is to learn a representation of words that are sufficiently present in the corpus to have a robust embedded representation.

❓ **Question** ❓ Learn a new `word2vec_3` model with a `min_count` higher than 5 (which is the default value) and a `word2vec_4` with a `min_count` smaller than 5, and then, compare the size of the vocabulary for all the different word2vecs that you have trained (you can choose any `vector_size` you want).

In [47]:
word2vec_3 = Word2Vec(sentences=X_train, vector_size=20, min_count=10)
wv_3 = word2vec_3.wv

In [48]:
len(wv_3.key_to_index)

4503

In [49]:
word2vec_4 = Word2Vec(sentences=X_train, vector_size=60, min_count=2)
wv_4 = word2vec_4.wv

In [50]:
len(wv_4.key_to_index)

16729

Remember that Word2Vec has an internal neural network that is optimized based on some predictions. These predictions actually correspond to predicting a word based on surrounding words. The surroundings words are in a `window` which corresponds to the number of words taken into account. And you can train the Word2Vec with different `window` sizes.

❓ **Question** ❓ Train a new `word2vec_5` model with a `window` different than previously (default is 5).

In [51]:
word2vec_5 = Word2Vec(sentences=X_train, vector_size=60, min_count=2, window=2)
wv_5 = word2vec_5.wv

In [52]:
len(wv_5.key_to_index)

16729

The arguments you have seen (`vector_size`, `min_count` and `window`) are usually the ones that you should start playing with to get a better performance for your model.

But you can also look at other arguments in the [**📚 Documentation - gensim.models.word2vec.Text8Corpus**](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Text8Corpus)

# Convert our train and test set to RNN-ready datasets

Remember that `Word2Vec` is the first step to the overall process of feeding such a representation into a RNN, as shown here:

<img src="word2vec_representation.png" width="400px" />



Now, let's work on Step 2 by converting the training and test data into their vector representation to be ready to be fed in RNNs.

❓ **Question** ❓ Now, write a function that, given a sentence, returns a matrix that corresponds to the embedding of the full sentence, which means that you have to embed each word one after the other and concatenate the result to output a 2D matrix (make sure that your output is a NumPy array)

❗ **Remark** ❗ You will probably notice that some words you are trying to convert throw errors as they are said not to belong to the dictionary:

- For the <font color=orange>test</font> set, this is understandable: <font color=orange>some words were not</font> in the <font color=blue>train</font> set and thus, their <font color=orange>embedded representation is unknown</font>
- for the <font color=blue>train set</font>, due to `min_count` hyperparameter, not all the words have a vector representation.

In any case, just skip the missing words here.

In [53]:
import numpy as np

example = ['this', 'movie', 'is', 'the', 'worst', 'action', 'movie', 'ever']
example_missing_words = ['this', 'movie', 'is', 'laaaaaaaaaame']

def embed_sentence(word2vec, sentence):
    embeddings = []
    
    for word in sentence:
        if word in word2vec.wv:
            embeddings.append(word2vec.wv[word])
            
    return np.array(embeddings)
### Checks
embedded_sentence = embed_sentence(word2vec, example)
assert(type(embedded_sentence) == np.ndarray)
assert(embedded_sentence.shape == (8, 100))

embedded_sentence_missing_words = embed_sentence(word2vec, example_missing_words)  
assert(type(embedded_sentence_missing_words) == np.ndarray)
assert(embedded_sentence_missing_words.shape == (3, 100))

❓ **Question** ❓ Write a function that, given a list of sentences (each sentence being a list of words/strings), returns a list of embedded sentences (each sentence is a matrix). Apply this function to the train and test sentences

💡 _Hint_: Use the previous function `embed_sentence`

In [54]:
def embedding(word2vec, sentences):
    return [embed_sentence(word2vec, sentence) for sentence in sentences]
    
X_train_embedded = embedding(word2vec, X_train)
X_test_embedded = embedding(word2vec, X_test)

❓ **Question** ❓ In order to have ready-to-use data, do not forget to pad your sequences so you have tensors which can be divided into batches (of `batch_size`) during the optimization. Store the padded values in `X_train_pad` and `X_test_pad`. Do not forget the important arguments of the padding ;)

In [55]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [56]:
max_length = max(len(sentence) for sentence in X_train_embedded)

X_train_pad = pad_sequences(X_train_embedded, maxlen=max_length, padding='post', dtype='float32')
X_test_pad = pad_sequences(X_test_embedded, maxlen=max_length, padding='post', dtype='float32')

assert(len(X_train_pad.shape) == 3)
assert(len(X_test_pad.shape) == 3)
assert(X_train_pad.shape[2] == 100)
assert(X_test_pad.shape[2] == 100)

In [57]:
X_train_pad.shape

(2500, 1082, 100)

In [59]:
X_test_pad.shape

(2500, 1082, 100)



🏁 Congratulations, you are now able to use `Word2Vec` to embed your words :)

💾 Don't forget to git add/commit/push your notebook...

🚀 ... and move on to the next challenge!
